In [1]:
import numpy as np
import sys
import os
from os.path import join as pjoin


# root_rot_velocity (B, seq_len, 1)
# root_linear_velocity (B, seq_len, 2)
# root_y (B, seq_len, 1)
# ric_data (B, seq_len, (joint_num - 1)*3)
# rot_data (B, seq_len, (joint_num - 1)*6)
# local_velocity (B, seq_len, joint_num*3)
# foot contact (B, seq_len, 4)
def mean_variance(data_dir, save_dir, joints_num): #相对表示的均值和方差
    file_list = os.listdir(data_dir)
    data_list = []

    for file in file_list:
        data = np.load(pjoin(data_dir, file))
        if np.isnan(data).any():
            print(file)
            continue
        data_list.append(data)

    data = np.concatenate(data_list, axis=0)
    print(data.shape)
    Mean = data.mean(axis=0)
    Std = data.std(axis=0)
    Std[0:1] = Std[0:1].mean() / 1.0
    Std[1:3] = Std[1:3].mean() / 1.0
    Std[3:4] = Std[3:4].mean() / 1.0
    Std[4: 4+(joints_num - 1) * 3] = Std[4: 4+(joints_num - 1) * 3].mean() / 1.0
    Std[4+(joints_num - 1) * 3: 4+(joints_num - 1) * 9] = Std[4+(joints_num - 1) * 3: 4+(joints_num - 1) * 9].mean() / 1.0
    Std[4+(joints_num - 1) * 9: 4+(joints_num - 1) * 9 + joints_num*3] = Std[4+(joints_num - 1) * 9: 4+(joints_num - 1) * 9 + joints_num*3].mean() / 1.0
    Std[4 + (joints_num - 1) * 9 + joints_num * 3: ] = Std[4 + (joints_num - 1) * 9 + joints_num * 3: ].mean() / 1.0

    assert 8 + (joints_num - 1) * 9 + joints_num * 3 == Std.shape[-1]
    if os.path.exists(pjoin(save_dir, 'Mean.npy')):
        os.remove(pjoin(save_dir, 'Mean.npy'))
    if os.path.exists(pjoin(save_dir, 'Std.npy')):
        os.remove(pjoin(save_dir, 'Std.npy'))

    np.save(pjoin(save_dir, 'Mean.npy'), Mean)
    np.save(pjoin(save_dir, 'Std.npy'), Std)

    return Mean, Std

In [2]:
def mean_variance_raw(data_dir, save_dir): #全局位置的均值和方差
    file_list = os.listdir(data_dir)
    data_list = []

    for file in file_list:
        data = np.load(pjoin(data_dir, file))
        if np.isnan(data).any():
            print(file)
            continue
        data_list.append(data)

    data = np.concatenate(data_list, axis=0)
    print(data.shape)
    Mean = data.mean(axis=0)
    Std = data.std(axis=0)
    if os.path.exists(pjoin(save_dir, 'Mean_raw.npy')):
        os.remove(pjoin(save_dir, 'Mean_raw.npy'))
    if os.path.exists(pjoin(save_dir, 'Std_raw.npy')):
        os.remove(pjoin(save_dir, 'Std_raw.npy'))

    np.save(pjoin(save_dir, 'Mean_raw.npy'), Mean)
    np.save(pjoin(save_dir, 'Std_raw.npy'), Std)

    return Mean, Std

In [3]:
if __name__ == '__main__':
    data_dir = '/sata/public/yyqi/Dataset/OCEAN/new_joints/'
    save_dir = '/sata/public/yyqi/Dataset/OCEAN/'
    # mean, std = mean_variance(data_dir, save_dir, 22)
    mean, std = mean_variance_raw(data_dir,save_dir)

    data_dir_vec = '/sata/public/yyqi/Dataset/OCEAN/new_joint_vecs/'
    save_dir_vec = '/sata/public/yyqi/Dataset/OCEAN/'
    mean_vec, std_vec = mean_variance(data_dir_vec, save_dir_vec, 22)


    print(mean)
    print(std)

(1586222, 22, 3)
(1586222, 263)
[[-2.1891168e-03  8.2828665e-01  2.0979439e-01]
 [ 4.4058967e-02  7.3480076e-01  2.0165431e-01]
 [-5.0930835e-02  7.3009056e-01  2.0601837e-01]
 [-3.6650833e-03  9.2525560e-01  1.7912452e-01]
 [ 8.1450775e-02  4.2076468e-01  2.7139816e-01]
 [-1.0420749e-01  4.2282185e-01  2.7574775e-01]
 [ 2.5961974e-03  1.0373110e+00  1.8027899e-01]
 [ 6.9359846e-02  5.9470072e-02  2.3016195e-01]
 [-1.0823476e-01  7.1252704e-02  2.3242174e-01]
 [-1.5728285e-03  1.0706782e+00  2.0287034e-01]
 [ 8.0087766e-02  2.1060057e-02  3.1166068e-01]
 [-1.1742713e-01  3.0223856e-02  3.0900109e-01]
 [-9.0125222e-03  1.2612382e+00  2.1528916e-01]
 [ 3.9588887e-02  1.1806058e+00  2.1395408e-01]
 [-4.5672689e-02  1.1776284e+00  2.1121424e-01]
 [ 1.0752120e-03  1.3765157e+00  2.7553481e-01]
 [ 1.0977590e-01  1.2094653e+00  2.1604078e-01]
 [-1.1870043e-01  1.2081469e+00  2.2074978e-01]
 [ 1.7492208e-01  9.8810005e-01  2.3459975e-01]
 [-1.9782035e-01  1.0121869e+00  2.5220096e-01]
 [ 1.579

### Check if your data is correct. If it's aligned with the given reference, then it is right

In [3]:
abs(mean-reference1).sum()

NameError: name 'reference1' is not defined

In [ ]:
abs(std-reference2).sum()

0.0